In [ ]:
import pandas as pd
from glob import glob

#selecting for DIPC or RTD files
zcheck = 0
while (zcheck not in ["d","r"]):
    print('Enter "d" for DIPC files, "r" for RTD:')
    z = input()
    zcheck = z
print("running script... press [ctrl + c] to abort")

    
if z == "d":
    files = sorted(glob('D*'))
    summary = pd.concat((pd.read_csv(file, skipfooter=1).assign(filename = file)
          for file in files),ignore_index = True)
    print("columns: ",summary.columns)
    print("shape: ",summary.shape)
    
    summary['RESOURCE_NAME2'] = summary['RESOURCE_NAME'].copy()
    summary['YEAR'] = summary['filename'].str[7:11]
    summary['MONTH'] = summary['filename'].str[11:13]
    summary['DAY'] = summary['filename'].str[13:15]
    summary['HOUR'] =summary['filename'].str[15:17]
    name = "DIPC_"+ summary['YEAR'].iloc[0] + summary['MONTH'].iloc[0] +summary['DAY'].iloc[0] + summary['HOUR'].iloc[0] +"_"+summary['YEAR'].iloc[-1] + summary['MONTH'].iloc[-1] +summary['DAY'].iloc[-1] + summary['HOUR'].iloc[-1]+".csv"
    summary['HOUR'] = pd.to_numeric(summary['HOUR']) - 1
    summary['HOUR'] = summary['HOUR'].replace(-1,23)
    summary['HOUR2'] = summary['HOUR']+1
    print("columns updated.")
    print("columns: ",summary.columns)
    print("shape: ",summary.shape)
    print('warning: when "HOUR" column = 23 or "HOUR2" = 24, DATE should be previous day')
    
    summary.to_csv(name)
    print("csv file created. enter any key to close terminal")
    z = input()




else:
    files = sorted(glob('R*'))
    RTD = pd.concat((pd.read_csv(file, skipfooter=1).assign(filename = file)
          for file in files),ignore_index = True)
    print("columns: ",RTD.columns)
    print("shape: ",RTD.shape)
    RTD['RESOURCE_NAME2'] = RTD['RESOURCE_NAME'].copy()

    print("creating supplementary columns... ")
    RTD[['DATE', 'TIME']] = RTD['RUN_TIME'].str.split(' ', 1, expand=True) #split to date and time
    RTD[['DAY', 'MONTH','YEAR']] = RTD['DATE'].str.split('/',2, expand=True) #split to date and time
    RTD['DATEHOUR'] = RTD['filename'].str[4:14]
    RTD['TIME'] = RTD['TIME'].fillna('12:00:00 AM') #fill 12:00:00 no values
    RTD[['HOUR', 'MINSECAMPM']] = RTD['TIME'].str.split(':', 1, expand=True) #split to hour and min
    RTD[['MINSEC', 'AMPM']] = RTD['MINSECAMPM'].str.split(' ', 1, expand=True) #split to get AMP/PM
    RTD['HOUR'] = pd.to_numeric(RTD['HOUR']) #convert to int
    RTD['HOUR'] = RTD.apply(lambda x:x['HOUR']+12 if x['AMPM']== 'PM' else x['HOUR'], axis=1) #convert PM dates to 24hour format
    RTD['HOUR'] = RTD['HOUR'].replace(12,0) #replace 12:00(from previous day) to zero hour
    RTD['HOUR'] = RTD['HOUR'].replace(24,12) #replace 24:00(12pm +12) back to 12th hour
    RTD['HOUR2'] = RTD['HOUR']+1
    print("columns updated")
    print("columns: ",RTD.columns)
    print("shape: ",RTD.shape)
#     summary['YEAR'] = summary['filename'].str[4:8]
#     summary['MONTH'] = summary['filename'].str[8:10]
#     summary['DAY'] = summary['filename'].str[10:12]
#     summary['HOUR'] =summary['filename'].str[12:14]
    name = "RTD_"+ RTD['DATEHOUR'].iloc[0] +"_" + RTD['DATEHOUR'].iloc[-1] +".csv"
    
    RTD.to_csv(name)
    print("csv file created. enter any key to close terminal")
    z = input()